### Loading a llm survey pdf (RAG System) - Testing llama3.1 model

In [72]:
from langchain_groq import ChatGroq # Inference
from langchain_community.document_loaders import PyPDFLoader # For loading pdf
from langchain.chains.summarize import load_summarize_chain # Summarization
from langchain.chains.question_answering import load_qa_chain # Q&A
from dotenv import load_dotenv # Detecting env variables
from langchain_pinecone import PineconeVectorStore # Vector Db
from langchain.text_splitter import RecursiveCharacterTextSplitter # Chunking
from langchain_community.embeddings import OllamaEmbeddings # Vector Embeddings
import os
from pprint import PrettyPrinter

In [56]:
load_dotenv()

True

In [57]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['PINECONE_API_KEY'] = os.getenv('PINECONE_API_KEY')

Loading the pdf

In [58]:
data = PyPDFLoader('R:\\Projects\\Langchain\\llmsurvey\\LLM Survey.pdf')

In [59]:
surveydata = data.load()

In [60]:
surveydata

[Document(page_content='PREPRINT 1\nA Comprehensive Overview of\nLarge Language Models\nHumza Naveed1, Asad Ullah Khan1,∗, Shi Qiu2,∗, Muhammad Saqib3,4,∗,\nSaeed Anwar5,6, Muhammad Usman5,6, Naveed Akhtar7, Nick Barnes8, Ajmal Mian9\n1University of Engineering and Technology (UET), Lahore, Pakistan\n2The Chinese University of Hong Kong (CUHK), HKSAR, China\n3University of Technology Sydney (UTS), Sydney, Australia\n4Commonwealth Scientific and Industrial Research Organisation (CSIRO), Sydney, Australia\n5King Fahd University of Petroleum and Minerals (KFUPM), Dhahran, Saudi Arabia\n6SDAIA-KFUPM Joint Research Center for Artificial Intelligence (JRCAI), Dhahran, Saudi Arabia\n7The University of Melbourne (UoM), Melbourne, Australia\n8Australian National University (ANU), Canberra, Australia\n9The University of Western Australia (UWA), Perth, Australia\nAbstract —\nLarge Language Models (LLMs) have recently demonstrated\nremarkable capabilities in natural language processing tasks and\n

Chunking

In [61]:
chunking = RecursiveCharacterTextSplitter(chunk_size=4000,chunk_overlap=0)
chunkdata = chunking.split_documents(surveydata)

In [62]:
len(chunkdata)

96

Vector Stores

In [63]:
index_name = "langchain3"

In [65]:
vectors = PineconeVectorStore.from_documents(chunkdata[:15],OllamaEmbeddings(model='mxbai-embed-large'),index_name=index_name)

In [16]:
vectors

Performed vector embeddings using OllamaEmbeddings and stored the embeddings to pinecone vector database.

Similarity Search

In [69]:
query = "What is transformers"

In [70]:
similarity = vectors.similarity_search(query=query)

In [74]:
similarity

[Document(page_content='PREPRINT 5\n2. Relative: To pass the information on the relative depen-\ndencies of different tokens appearing at different locations in\nthe sequence, a relative positional encoding is calculated by\nsome kind of learning. Two famous types of relative encodings\nare:\nAlibi: [65] In this approach, a scalar bias is subtracted from\nthe attention score calculated using two tokens which increases\nwith the distance between the positions of the tokens. This\nlearned approach effectively favors using recent tokens for\nattention.\nRoPE: Keys, queries, and values are all vectors in the LLMs.\nRoPE [66] involves the rotation of the query and key represen-\ntations at an angle proportional to their absolute positions of\nthe tokens in the input sequence. This step results in a relative\npositional encoding scheme which decays with the distance\nbetween the tokens.\nE. Activation Functions\nThe activation functions serve a crucial role in the curve-\nfitting abilities o

Instancing the model

In [75]:
groqllm = ChatGroq(model='Llama-3.1-70b-Versatile',temperature=0)

Summarization

In [76]:
chain = load_summarize_chain(groqllm,chain_type='map_reduce',verbose=True)
chain.run(chunkdata[:5])



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"PREPRINT 1
A Comprehensive Overview of
Large Language Models
Humza Naveed1, Asad Ullah Khan1,∗, Shi Qiu2,∗, Muhammad Saqib3,4,∗,
Saeed Anwar5,6, Muhammad Usman5,6, Naveed Akhtar7, Nick Barnes8, Ajmal Mian9
1University of Engineering and Technology (UET), Lahore, Pakistan
2The Chinese University of Hong Kong (CUHK), HKSAR, China
3University of Technology Sydney (UTS), Sydney, Australia
4Commonwealth Scientific and Industrial Research Organisation (CSIRO), Sydney, Australia
5King Fahd University of Petroleum and Minerals (KFUPM), Dhahran, Saudi Arabia
6SDAIA-KFUPM Joint Research Center for Artificial Intelligence (JRCAI), Dhahran, Saudi Arabia
7The University of Melbourne (UoM), Melbourne, Australia
8Australian National University (ANU), Canberra, Australia
9The University of Western Australia (UWA), Perth, Australia
Abstract —
Large L

'Here is a concise summary of the text:\n\nLarge Language Models (LLMs) have made significant progress in natural language processing tasks, but also come with computational costs and limitations. Researchers have explored methods to address these limitations, such as parameter-efficient tuning and knowledge distillation. The field is shifting towards instruction-tuned and open-source models, with notable models including GPT-3, T5, and mT5. LLMs have demonstrated emergent abilities like reasoning and decision-making, and are being applied in various settings, including multi-modal and robotics applications. This article provides a comprehensive overview of LLM research, covering architecture, training, evaluation, applications, and challenges.'

It gave good summarization of the LLM Survey Pdf.

Question and Answering

In [77]:
question_answering = load_qa_chain(groqllm,chain_type='map_reduce',verbose=True)

In [78]:
question_answering.run(question=query, input_documents = chunkdata[:5])



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text verbatim.
______________________
PREPRINT 1
A Comprehensive Overview of
Large Language Models
Humza Naveed1, Asad Ullah Khan1,∗, Shi Qiu2,∗, Muhammad Saqib3,4,∗,
Saeed Anwar5,6, Muhammad Usman5,6, Naveed Akhtar7, Nick Barnes8, Ajmal Mian9
1University of Engineering and Technology (UET), Lahore, Pakistan
2The Chinese University of Hong Kong (CUHK), HKSAR, China
3University of Technology Sydney (UTS), Sydney, Australia
4Commonwealth Scientific and Industrial Research Organisation (CSIRO), Sydney, Australia
5King Fahd University of Petroleum and Minerals (KFUPM), Dhahran, Saudi Arabia
6SDAIA-KFUPM Joint Research Center for Artificial Intelligence (JRCAI), Dhahran, Saudi Arabia
7The University of Melbourne (UoM), Melbourne, Australia
8Australi

'Transformers are a type of neural network architecture that was introduced in a 2017 research paper by Vaswani et al. They are primarily used for natural language processing (NLP) tasks, such as language translation, text summarization, and text generation.\n\nTransformers rely on self-attention mechanisms to process input sequences, allowing them to capture long-range dependencies and contextual relationships between different parts of the input. They are particularly well-suited for tasks that require parallelization and attention mechanisms.\n\nTransformers have become a fundamental component of many modern NLP models, including large language models (LLMs), and have achieved state-of-the-art results in many applications.'

It gave proper answer for the given question. Tested the new llama3.1 70b Versatile model.